# Modelo proposto por Lee e Cho (2004)

Modelo adaptado para utilizar o custo da árvoare ao invés de utilizar o número de arestas como parâmetro.



$$ Maximizar \quad Z$$
$$\mbox{Subject to:}$$

$$\sum_{i \in V}{x_{ir_k}^{kd}} - \sum_{i \in V}{x_{r_{k}i}^{kd}} = -1\quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,d) \in E}{x_{id}^{kd}} - \sum_{(d,i) \in E}{x_{di}^{kd}} = 1 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ij}^{kd}} - \sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ji}^{kd}} = 0 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$x_{ij}^{kd} \leq y_{ij}^{k} \quad \forall k \in K, d \in D^k  \text{ e }  \forall (i,j) \in E$$

$$b_{ij}  - \sum_{k \in K} y_{ij}^{k} t^k >= Z \quad \forall (i,j) \in E$$


$$ \sum_{(i,j) \in E} y_{ij}^{k} * custo_{ij} <= OPT^{k} \quad \forall k \in K$$


$$x_{ij}^{kd} \in \lbrace 0, 1 \rbrace , y_{ij}^{k} \in \lbrace 0, 1 \rbrace  {{\forall k \in K,  \forall d \in D^k} \atop \forall (i,j) \in E}$$

## Lendo os dados da instância

In [121]:
from classes import Network, MulticastGroup
from multicastpacking import MulticastPacking, solver
import reader

#instancen of the problem
file = "../../../MPP_instances/n120/b120_1.brite"

# listofopt = "..."

links = reader.get_network (file)
net = Network (links, nodes = 120)


mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

problem = MulticastPacking (net, mgroups)

KSIZE = len(problem.groups)+1
NODES = net.nodes+1

# Definição do Modelo

## Variáveis

$ x_{ij}^{kd} $ e $ y_{ij}^{k} $

In [122]:
from gurobipy import *

m = Model ("MPP by Cost")

var_x = {}
var_y = {}

for link in net.links:
    for k in xrange (1, KSIZE):
        for d in problem.groups[k-1].members:
            x=link[0],link[1],k,d,
            y=link[1],link[0],k,d,
            var_x[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
            var_x[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            

for k in xrange (1, KSIZE):
    for link in net.links:
        x=link[0],link[1],k
        y=link[1],link[0],k
        var_y[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
        var_y[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

m.update ()

Z = m.addVar(vtype=GRB.INTEGER, name="Z", obj=1)
m.update ()

## Fluxo 1

In [123]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',k,d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,k,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',k,d) )
            == -1,
            name=str(_name)
        )

m.update ()

## Fluxo 2

In [124]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow2',k,d,j,       
            m.addConstr (
                quicksum(
                   var_x[x] for x in tuplelist (var_x).select ('*',j,k,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',k,d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

m.update ()

## Fluxo 3

In [125]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow3',k,d
        m.addConstr (
            quicksum (
                var_x[x] for x in tuplelist (var_x).select ('*',d,k,d)
            )
            -
            quicksum (
                var_x[x] for x in tuplelist (var_x).select (d,'*',k,d)
            )
            == 1,
            name=str(_name)
        )
        
m.update ()

## Restrição que Força $y_{ij}^{k}$ ser igual 1 se a aresta $(i,j)$ é usada por alguma árvore:

$$x_{ij}^{kd} \leq y_{ij}^{k} \quad \forall k \in K, d \in D^k  \text{ e }  \forall (i,j) \in E$$

In [126]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],k,d,
            y=link[0],link[1],k
            _name='mark',link[0],link[1],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],k,d,
            y=link[1],link[0],k
            _name='mark',link[1],link[0],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )
m.update ()

## Restrição de Capacidade

In [127]:
# constr = []

# for link in net.links:    
#     _name='capacity',link
#     constr.append(
#         m.addConstr (
#             links[link][1]
#             -
#             quicksum (
#                 (var_y[ (x[0],x[1],x[2]) ] + var_y[ (x[1],x[0],x[2]) ] )*problem.groups[x[2]-1].traffic
#                     for x in tuplelist (var_y).select (link[1],link[0],'*')
#             )
#             >= Z,
#             name=str(_name)
#         )
#     )
# m.update ()

for k in xrange(1,KSIZE):
    for link in net.links:
        _name='cap',k,link[0],link[1]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[0],link[1],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
        _name='cap',k,link[1],link[0]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[1],link[0],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
m.update ()  

## Restrição de Custo Ótimo

In [128]:
list = [7026.0 , 6805.0 , 7498.0 , 7364.0 , 8145.0]

def get_cost (net_, l):
    if l in net_.links:
        return net_.properties (l)[0]
    else:
        return net_.properties ( (l[1],l[0]) )[0]

    
LeeConstr = []
for k in xrange(1, KSIZE):
    
    LeeConstr.append (
        m.addConstr (    
            quicksum (
                var_y[y] * get_cost (net, (y[0],y[1]) )
                for y in tuplelist(var_y).select ('*','*',k)            
            )
            <= 1.5*list[k-1],
            name=str('kk')
        )
    )
m.update ()

## Função Objetivo

In [129]:
m.setObjective (Z, GRB.MAXIMIZE)
m.update ()

In [141]:
for x in xrange(1,len(LeeConstr)):    
    LeeConstr[x-1].setAttr (GRB.attr.RHS, 1.2*list[x-1])

m.update ()

m.write ('teste.lp')

In [142]:
m.optimize ()

Optimize a model with 55381 rows, 44641 columns and 185760 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+03]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+04]
Presolve removed 2289 rows and 119 columns
Presolve time: 0.45s
Presolved: 53092 rows, 44522 columns, 173390 nonzeros

Loaded MIP start with objective 26

Variable types: 0 continuous, 44522 integer (44521 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   15230    5.0999840e+01   3.560458e+02   0.000000e+00      5s
   16746    5.1000000e+01   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 5.100000e+01, 16746 iterations, 5.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   51.00000    0  251   26.00000   51.00000  96.2%     -    8s
H    0     0                      45.0000000   5

In [91]:
print m.ObjVal

GurobiError: Unable to retrieve attribute 'ObjVal'